In [1]:
import torch
from torch import nn
from d2l import torch as d2l

# 图像卷积

# 互相关计算

In [3]:
def corr2d(X, K):
    h, w = K.shape
    """比如说K 3x3 , X 16x16 , 那Y 16-3+1=14 """
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j: j+w] * K).sum() # 很清楚了，这就是卷积！
    return Y

# 实现二维卷积层

In [4]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

## 卷积层的一个简单应用：检测图像中不同颜色的边缘

In [5]:
X = torch.ones((6,8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [7]:
K = torch.tensor([[1.0, -1.0]])
# 两个元素没有变的话，输出就是一个0，

## 输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘

In [9]:
Y = corr2d(X, K)
Y, Y.shape

(tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]),
 torch.Size([6, 7]))

## 卷积核K只可以检测垂直边缘

In [11]:
X.t(), corr2d(X.t(), K) # 转置就不行了

(tensor([[1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1.]]),
 tensor([[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]))

# 学习由X生成Y的卷积核

In [76]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias = False)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2 # ！！！这个平方很好玩，改成其他的就不work了   哈哈哈 其他的超参也很好玩
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad # 3e-2 是学习率
    if (i+1) % 2 == 0:
        print(f'batch{i+1}, loss {l.sum():.3f}')

batch2, loss 3.321
batch4, loss 0.593
batch6, loss 0.114
batch8, loss 0.025
batch10, loss 0.007


## 所学习的卷积核的权重张量

In [77]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9974, -0.9841]])